# CQW - Medium Reports
Updated: 2019-10-16

Medium reports can run for upwards of 60 seconds depending on load on the system. The following reports are found in this notebook. 

  - Report 2a4  - Sales (Wholesale) by Stores in Tennessee
  - Report 2a4b - Sales (Net) By Stores in Alabama 
  - Report 2a5  - Sales (Net) By Promotion 
  - Report 2a7  - Sales (Net) By Store in South Dakota
  - Report 2a7b - Sales (Net) With Coupons By Stores in South Dakota
  - Report 2a8  - Sales by Promotion
  - Report 2d1  - Sales of Specific Stores in 2000, 2001, 2002
  - Report 2e1  - Sales by all Stores, all dates
  - Report 2e2  - Sales by Top 10 stores, all dates

### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

### Create a Function for Running the Queries
This function is used to time the queries and return the total time taken. The parameter is the SQL that will be run and the function will execute against the row-based and col-based tables. 

In [ ]:
def runquery(sql):
    
    %sql -q SET CURRENT SCHEMA = CQWROW
        
    print("Starting execution of ROW query")    
    start_time = time.time()

    %sql -q {sql}
    
    end_time = time.time()
    row = end_time - start_time
    
    print("Row run time = {0}".format(row))
    
    %sql -q SET CURRENT SCHEMA = CQWCOL
    
    print()
    print("Starting execution of COLUMNAR query")        
    
    start_time = time.time()

    results = %sql -q {query}
    
    end_time = time.time()
    col = end_time - start_time
    
    print("Column run time = {0}".format(col))    
    
    pdisplay(results)
    
    %sql -bar VALUES ('Row',:row),('Column',:col)

## Sales (Wholesale) by Stores in Tennessee

In [ ]:
query = '''
SELECT
    "STORE"."S_STORE_SK" AS "Stores_S_STORE_SK", 
    SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "WholeSale_Cost__Store_Sales_"
FROM
    "STORE" "STORE"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
WHERE 
    "STORE"."S_COUNTRY" IN ( 
        'United States' ) AND
    "STORE"."S_STATE" IN ( 
        'TN' ) 
GROUP BY 
    "STORE"."S_STORE_SK"
'''
runquery(query)

## Sales (Net) By Stores in Tennessee 

In [ ]:
query = '''
SELECT
    "STORE"."S_STORE_SK" AS "Stores_S_STORE_SK",
    SUM("STORE_SALES"."SS_NET_PROFIT") AS "Net_Profit__Stor", 
    SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "Quantity__Store_"
FROM
    "STORE" "STORE"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
WHERE 
    "STORE"."S_COUNTRY" IN ( 
        'United States' ) AND
    "STORE"."S_STATE" IN ( 
        'TN' ) 
GROUP BY 
    "STORE"."S_STORE_SK"
'''
runquery(query)

## Sales (Net) By Promotion 

In [ ]:
query = '''
SELECT
    "PROMOTION"."P_PROMO_SK" AS "Promotions_P_PROMO_SK", 
    SUM("STORE_SALES"."SS_NET_PAID") AS "Net_Paid__Store_Sales_"
FROM
    "PROMOTION" "PROMOTION"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "PROMOTION"."P_PROMO_SK" = "STORE_SALES"."SS_PROMO_SK" 
GROUP BY 
    "PROMOTION"."P_PROMO_SK"
'''
runquery(query)

## Sales (Net) By Store in Tennessee

In [ ]:
query = '''
SELECT
    "STORE"."S_STORE_SK" AS "Stores_S_STORE_SK", 
    SUM("STORE_SALES"."SS_NET_PAID") AS "Net_Paid__Store_Sales_"
FROM
    "STORE" "STORE"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
WHERE 
    "STORE"."S_COUNTRY" IN ( 
        'United States' ) AND
    "STORE"."S_STATE" IN ( 
        'TN' ) 
GROUP BY 
    "STORE"."S_STORE_SK"
'''
runquery(query)

## Sales (Net) With Coupons By Stores in Tennessee

In [ ]:
query = '''
SELECT
    "STORE"."S_STORE_SK" AS "Stores_S_STORE_SK", 
    SUM("STORE_SALES"."SS_NET_PAID") AS "Net_Paid__Store_Sales_",
    SUM("STORE_SALES"."SS_COUPON_AMT") AS "Coupon_Amount__S",
    SUM("STORE_SALES"."SS_NET_PAID" - CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "Gross_Profit__St"
FROM
    "STORE" "STORE"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
WHERE 
    "STORE"."S_COUNTRY" IN ( 
        'United States' ) AND
    "STORE"."S_STATE" IN (
        'TN' )
GROUP BY 
    "STORE"."S_STORE_SK"
'''
runquery(query)

## Sales by Promotion

In [ ]:
query = '''
SELECT
    "PROMOTION"."P_PROMO_SK" AS "Promotions_P_PROMO_SK", 
    SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "WholeSale_Cost__Store_Sales_"
FROM
    "PROMOTION" "PROMOTION"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "PROMOTION"."P_PROMO_SK" = "STORE_SALES"."SS_PROMO_SK" 
GROUP BY 
    "PROMOTION"."P_PROMO_SK"
'''
runquery(query)

## Sales of Specific Stores in 2000, 2001, 2002

In [ ]:
query = '''
SELECT
    AVG(CAST("STORE_SALES"."SS_LIST_PRICE" AS DOUBLE PRECISION)) AS "List_Price__Store_Sales_", 
    "STORE"."S_STORE_SK" AS "S_STORE_SK"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK"
            INNER JOIN "STORE" "STORE"
            ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2002, 
        2001, 
        2000 ) AND
    ("STORE"."S_STORE_SK" IN ( 
356, 194, 138, 505, 171, 26, 31, 861, 729, 208,
842, 117, 837, 49, 1002, 663, 232, 539, 46, 966,
224, 98, 58, 545, 470, 512, 790, 829, 888, 240,
222, 187, 764, 695, 411, 215, 942, 719, 131, 305,
850, 258, 248, 303, 610, 55, 379, 348, 900, 978,
806, 681, 202, 402, 894, 634, 94, 388, 165, 582,
366, 899, 184, 629, 207, 979, 974, 328, 436, 592,
167, 596, 991, 728, 390, 904, 201, 389, 265, 771,
688, 385, 570, 867, 955, 954, 831, 157, 80, 193,
135, 767, 78, 741, 907, 736, 214, 849, 492, 488,
843, 371, 740, 859, 617, 876, 155, 291, 776, 210,
500, 313, 433, 270, 645, 199, 140, 796, 988, 789,
231, 15, 662, 908, 554, 910, 496, 874, 678, 647,
339, 846, 753, 538, 136, 644, 151, 868, 997, 491,
76, 494, 845, 115, 1000, 750, 864, 784, 605, 929,
16, 895, 658, 228, 154, 945, 893, 479, 1, 732,
694, 416, 281, 692, 716, 373, 102, 972, 441, 977,
340, 192, 668, 383, 944, 498, 52, 33, 935, 799,
563, 253, 96, 683, 65, 118, 851, 404, 296, 251,
397, 552, 625, 992, 674, 930, 304, 169, 873, 504,
926, 558, 905, 456, 603, 660, 85, 133, 548, 183,
727, 17, 526, 722, 308, 204, 733, 631, 414, 761,
317, 375, 777, 203, 180, 797, 690, 161, 473, 45,
106, 933, 575, 283, 162, 682, 599, 531, 218, 875,
774, 702, 511, 869, 863, 881, 836, 175, 950, 81 )
OR
    "STORE"."S_STORE_SK" IN ( 
443, 871, 779, 723, 345, 549, 464, 983, 424, 601,
261, 680, 104, 362, 25, 769, 309, 999, 998, 880,
768, 959, 840, 497, 260, 334, 847, 34, 59, 825,
981, 938, 885, 807, 484, 423, 943, 471, 925, 255,
781, 275, 725, 311, 178, 717, 43, 95, 307, 408,
91, 141, 766, 758, 745, 418, 770, 299, 39, 179,
316, 714, 395, 520, 507, 805, 844, 205, 61, 862,
110, 335, 426, 126, 478, 243, 306, 756, 622, 973,
919, 803, 272, 693, 649, 987, 785, 529, 608, 485,
226, 934, 367, 197, 166, 962, 564, 30, 685, 918,
640, 802, 9, 735, 361, 347, 546, 854, 109, 12,
220, 630, 713, 449, 763, 177, 923, 534, 743, 642,
160, 369, 92, 439, 344, 477, 442, 124, 616, 557,
493, 75, 229, 754, 521, 752, 354, 293, 377, 936,
911, 652, 93, 834, 664, 186, 391, 447, 376, 696,
145, 134, 301, 314, 581, 975, 359, 227, 64, 217,
434, 963, 351, 517, 650, 35, 906, 574, 461, 410,
646, 129, 113, 532, 407, 898, 527, 435, 462, 780,
724, 238, 673, 346, 274, 44, 130, 523, 295, 246,
619, 920, 360, 73, 419, 996, 219, 537, 446, 556,
22, 252, 355, 980, 739, 216, 816, 271, 487, 562,
555, 525, 357, 237, 18, 142, 331, 986, 870, 70,
353, 310, 632, 578, 417, 363, 466, 903, 798, 315,
641, 4, 989, 489, 533, 689, 726, 86, 547, 501,
119, 827, 37, 621, 794, 889, 10, 242, 901, 952 ) 
OR
    "STORE"."S_STORE_SK" IN ( 
917, 760, 319, 559, 195, 565, 68, 499, 838, 902,
514, 112, 393, 990, 327, 971, 279, 970, 282, 751,
460, 535, 427, 233, 320, 341, 42, 444, 56, 290,
99, 32, 639, 148, 465, 190, 879, 928, 852, 300,
191, 585, 508, 937, 749, 890, 259, 430, 415, 587,
146, 273, 502, 584, 72, 495, 196, 913, 269, 8,
731, 23, 602, 262, 615, 788, 50, 707, 451, 513,
655, 667, 730, 748, 588, 198, 69, 783, 188, 666,
422, 100, 121, 553, 704, 857, 968, 747, 406, 965,
287, 336, 241, 490, 995, 577, 896, 976, 755, 88,
572, 173, 168, 421, 737, 822, 897, 221, 374, 84,
412, 543, 860, 263, 600, 964, 384, 289, 89, 40,
60, 28, 19, 810, 957, 256, 804, 823, 734, 568,
811, 541, 636, 127, 107, 503, 365, 48, 74, 775,
182, 103, 882, 149, 635, 116, 886, 516, 939, 773,
941, 225, 697, 567, 2, 651, 468, 542, 482, 712,
922, 463, 114, 13, 20, 765, 583, 791, 152, 401,
705, 793, 476, 528, 676, 698, 914, 985, 469, 700,
510, 590, 607, 509, 277, 486, 711, 958, 139, 298,
669, 144, 24, 440, 235, 53, 759, 924, 961, 595,
63, 795, 561, 101, 432, 445, 819, 51, 832, 452,
594, 623, 267, 721, 746, 122, 474, 818, 808, 266,
560, 718, 79, 398, 706, 212, 108, 858, 472, 381,
66, 82, 826, 213, 569, 77, 150, 744, 812, 536,
589, 659, 960, 392, 11, 329, 591, 626, 782, 579 )
OR
    "STORE"."S_STORE_SK" IN ( 
814, 387, 519, 687, 637, 159, 455, 566, 396, 429,
480, 720, 835, 123, 946, 321, 916, 450, 515, 247,
14, 872, 677, 593, 350, 654, 323, 686, 833, 672,
158, 90, 264, 638, 648, 41, 438, 956, 947, 457,
614, 691, 318, 670, 671, 800, 403, 83, 967, 250,
627, 5, 413, 400, 643, 518, 540, 609, 372, 801,
786, 67, 787, 185, 481, 285, 420, 3, 343, 742,
230, 571, 326, 993, 830, 467, 97, 330, 824, 853,
921, 47, 57, 454, 586, 453, 223, 431, 701, 865,
877, 125, 333, 628, 927, 189, 244, 475, 54, 337,
370, 708, 302, 409, 143, 1001, 762, 778, 949, 399,
349, 699, 883, 624, 288, 620, 284, 211, 524, 633,
932, 948, 940, 137, 358, 665, 679, 71, 573, 684,
425, 839, 915, 448, 325, 172, 234, 153, 709, 891,
386, 324, 338, 394, 884, 661, 856, 297, 286, 576,
613, 132, 828, 174, 332, 550, 931, 612, 62, 912,
156, 544, 105, 209, 128, 278, 280, 551, 6, 7, 38,
675, 909, 111, 27, 809, 294, 738, 342, 36, 892,
292, 982, 866, 239, 598, 322, 951, 245, 29, 855,
312, 606, 176, 382, 249, 506, 618, 817, 428, 200,
653, 181, 206, 969, 994, 21, 878, 254, 772, 953,
580, 164, 163, 437, 120, 656, 792, 841, 848, 611,
887, 380, 815, 170, 268, 710, 715, 352, 522, 87,
236, 530, 597, 483, 813, 821, 364, 984, 368, 147,
459, 458, 276, 757, 257, 405, 820, 604, 657 ) 
OR
    "STORE"."S_STORE_SK" IN ( 
        378, 
        703, 
        -99999 )) 
GROUP BY 
    "STORE"."S_STORE_SK"
'''
runquery(query)

##  Sales by All Stores, All Dates

In [ ]:
query = '''
SELECT
    "STORE"."S_STORE_SK" AS "Stores_S_STORE_SK", 
    AVG(CAST("STORE_SALES"."SS_LIST_PRICE" AS DOUBLE PRECISION)) AS "List_Price__Store_Sales_"
FROM
    "STORE" "STORE"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
GROUP BY 
    "STORE"."S_STORE_SK"
'''
runquery(query)

##  Sales by Top 10 Stores, All Dates

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
    "STORE"."S_STORE_SK" AS "Stores_S_STORE_SK", 
    AVG(CAST("STORE_SALES"."SS_LIST_PRICE" AS DOUBLE PRECISION)) AS "List_Price__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK"
            INNER JOIN "STORE" "STORE"
            ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
WHERE 
    "STORE"."S_STORE_SK" IN ( 
        986, 
        842, 
        625, 
        247, 
        949, 
        8, 
        775, 
        572, 
        547, 
        265 ) 
GROUP BY 
    "DATE_DIM"."D_YEAR", 
    "STORE"."S_STORE_SK"
'''
runquery(query)

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]